In [9]:
using Distributed, Pkg
Pkg.activate(".")
withenv("JULIA_PROJECT"=>pwd()) do
    addprocs(4)
end

Activating environment at `C:\Users\carsten\Desktop\Oulu2020\backup\distributed_fileloading\Project.toml`


4-element Array{Int64,1}:
 2
 3
 4
 5

In [10]:
@everywhere using DistributedArrays
@everywhere using Serialization

In [11]:
# create fake data
using Serialization
for i in 1:4
    serialize("$i.bin", rand(1000)) # 8 GB chunks
end

In [12]:
# load in parallel
fp = filter(x -> endswith(x, ".bin"), readdir())

4-element Array{String,1}:
 "1.bin"
 "2.bin"
 "3.bin"
 "4.bin"

In [13]:
dfp = distribute(fp)

4-element DArray{String,1,Array{String,1}}:
 "1.bin"
 "2.bin"
 "3.bin"
 "4.bin"

In [14]:
xs = map(deserialize, dfp);

In [17]:
@everywhere println(localindices($xs))

(1:0,)
      From worker 2:	(1:1,)
      From worker 4:	(3:3,)
      From worker 5:	(4:4,)
      From worker 3:	(2:2,)


In [61]:
map(x->sum(cos.(x).^2), xs)

4-element DArray{Float64,1,Array{Float64,1}}:
 729.5141517755486
 722.0139261917391
 712.6153786929397
 728.0392946509859